# SQL agent

## Setup

### Imports

In [15]:
import os
import re
import pickle
from typing import (
    List,
    Any
)

import db_connect
from test_data import TestData
from prompt import (
    PREFIX,
    SUFFIX,
    TABLE_DESCRIPTIONS
)

from langsmith import Client

from langchain_core.language_models import BaseLanguageModel

from langchain.prompts import PromptTemplate

from langchain.agents.agent_types import AgentType
from langchain.agents.agent_toolkits.sql import base
from langchain.agents.agent import AgentExecutor
from langchain.agents import create_openai_tools_agent, create_react_agent, create_sql_agent
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.tools.sql_database import tool as sql_tools

from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain_experimental.llms.ollama_functions import (
    OllamaFunctions,
    DEFAULT_RESPONSE_FUNCTION
)

from langchain_core.output_parsers import StrOutputParser

from langchain_core.messages.human import HumanMessage
from langchain_core.messages.ai import AIMessage

from langchain_experimental.llms.ollama_functions import convert_to_ollama_tool
from langchain.schema.agent import AgentFinish

from langchain_core.pydantic_v1 import BaseModel, Field

from langchain.tools import (tool, BaseTool)

from prompt_generator import PromptGenerator
from examples import FEW_SHOT_EXAMPLES

from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain.agents.format_scratchpad.openai_functions import (
    format_to_openai_function_messages,
)
from langchain.agents.output_parsers.openai_functions import (
    OpenAIFunctionsAgentOutputParser,
)

from langchain_core.runnables.config import RunnableConfig
from langchain.callbacks.tracers import ConsoleCallbackHandler

from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

### LangSmith

In [16]:
os.environ["LANGCHAIN_PROJECT"] = "text2sql"
client = Client()

### Load models

In [17]:
llama3_embeddings = OllamaEmbeddings(model="llama3:8b", temperature=0)

llama3 = Ollama(model="llama3:instruct", temperature=0)
llama3_q8 = Ollama(model="llama3:8b-instruct-q8_0", temperature=0)

llama3__with_functions = OllamaFunctions(model="llama3:instruct", format="json")
llama3_q8__with_functions = OllamaFunctions(model="llama3:8b-instruct-q8_0", format="json")

### Connect to DB with Readonly role

In [18]:
db = db_connect.get_db()

#### Check connection

In [19]:
db.run("select * from passenger")

"[(1, 'John'), (2, 'James'), (3, 'Poul'), (4, 'Christofer'), (5, 'Superman'), (6, 'Donald'), (7, 'Douglas'), (8, 'Dwight'), (9, 'Earl'), (10, 'Edgar'), (11, 'Edmund'), (12, 'Edwin'), (13, 'Elliot'), (14, 'Eric'), (15, 'Ernest'), (16, 'Ethan'), (17, 'Ezekiel'), (18, 'Felix'), (19, 'Franklin'), (20, 'Frederick'), (21, 'Gabriel'), (22, 'Joseph'), (23, 'Joshua'), (24, 'Julian'), (25, 'Alice'), (26, 'Bob'), (27, 'Charlie'), (28, 'David'), (29, 'Emily'), (30, 'Frank'), (31, 'George'), (32, 'Helen'), (33, 'Irene'), (34, 'Jack'), (35, 'Kate'), (36, 'Leo'), (37, 'Mary'), (38, 'Nancy'), (39, 'Oliver'), (40, 'Paul'), (41, 'Qiana'), (42, 'Robert'), (43, 'Samantha'), (44, 'Thomas'), (45, 'Victoria')]"

-----

## Create agent

### Prompt and error handler

In [20]:
prompt_generator = PromptGenerator().set_example_selector(
        examples=FEW_SHOT_EXAMPLES,
        embedding_llm=llama3_embeddings,
        k=3
)

### Create functions for `OllamaFunctions`

#### Pydentic classes (Functions)

In [21]:
class QuerySQLDatabaseTool(BaseModel):
    """INPUT to this tool is a DETAILED and CORRECT SQL query, 
    OUTPUT is a RESULT FROM the DATABASE. If the query is not 
    correct, an error message will be returned. If an error is 
    returned, rewrite the query, check the query, and try again. 
    If you encounter an issue with Unknown column 'xxxx' in 
    'field list', use sql_db_schema to query the correct table fields."""
    query: str = Field(
        description="Detailed and correct SQL query."
    )


class InfoSQLDatabaseTool(BaseModel):
    """INPUT to this tool is a comma-separated LIST OF TABLES, 
    output is the schema and sample rows for those tables. 
    Be sure that the tables actually exist by calling 
    sql_db_list_tables first! Example Input: table1, table2, table3.
    This tool will not help if you need to get information about a column!"""
    input_list: str = Field(
        description="Comma-separated list of tables. Example: table1, table2, table3"
    )


class ListSQLDatabaseTool(BaseModel):
    """INPUT to this tool is a EMPTY STRING, 
    OUTPUT is a comma-separated LIST OF TABLES 
    in database. Use this tool to select the 
    tables needed to respond to the user."""
    empty_string: str = Field(description="Empty string")


class QuerySQLCheckerTool(BaseModel):
    """INPUT to this tool is a QUERY to check, 
    OUTPUT is a CORRECT QUERY in database. 
    Use this tool to double check if 
    your query is correct before executing it. 
    Always use this tool before executing a 
    query with sql_db_query!"""
    query_to_check: str = Field(
        description="SQL query that needs to be checked before execution."
    )
    

class ConversationalResponse(BaseModel):
    """INPUT to this tool is a message to 
    the user Use this tool to text replies 
    to the user or if no other tool is 
    suitable for you. For example, 
    if you are ready to answer a 
    question. Or if the user just 
    wants to chat."""
    message_to_user: str = Field(
        description="Message or response or answer to user."
    )



functions = [
    QuerySQLDatabaseTool, 
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    ConversationalResponse
]
functions = [convert_to_ollama_tool(f) for f in functions]
functions[-1]['name'] = DEFAULT_RESPONSE_FUNCTION['name']

#### Tools

In [22]:
@tool
def QuerySQLDatabaseTool(query: str):
    """INPUT to this tool is a DETAILED and CORRECT SQL query, 
    OUTPUT is a RESULT FROM the DATABASE. If the query is not 
    correct, an error message will be returned. If an error is 
    returned, rewrite the query, check the query, and try again. 
    If you encounter an issue with Unknown column 'xxxx' in 
    'field list', use sql_db_schema to query the correct table fields."""
    return db.run_no_throw(query)

@tool
def InfoSQLDatabaseTool(input_list: str):
    """INPUT to this tool is a comma-separated LIST OF TABLES, 
    output is the schema and sample rows for those tables. 
    Be sure that the tables actually exist by calling 
    sql_db_list_tables first! Example Input: table1, table2, table3.
    This tool will not help if you need to get information about a column!"""
    
    res = ""
    res += "company, " if "company" in input_list else ""
    res += "pass_in_trip, " if "pass_in_trip" in input_list else ""
    res += "passenger, " if "passenger" in input_list else ""
    res += "trip, " if "trip" in input_list else ""
    
    res = res[:-2]
    
    return db.get_table_info_no_throw(
            [t.strip() for t in res.split(",")]
        )

@tool
def ListSQLDatabaseTool(empty_string: str = ""):
    """INPUT to this tool is a EMPTY STRING, 
    OUTPUT is a comma-separated LIST OF TABLES 
    in database. Use this tool to select the 
    tables needed to respond to the user."""
    return ", ".join(db.get_usable_table_names())

QUERY_CHECKER = """
{query}
Double check the {dialect} query above for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

Output the final SQL query only.

SQL Query: """

checker_prompt = PromptTemplate.from_template(
    template=QUERY_CHECKER
)

@tool
def QuerySQLCheckerTool(query_to_check: str):
    """INPUT to this tool is a QUERY to check, 
    OUTPUT is a CORRECT QUERY in database. 
    Use this tool to double check if 
    your query is correct before executing it. 
    Always use this tool before executing a 
    query with sql_db_query!"""
    chain = checker_prompt | llama3 | StrOutputParser()
    return chain.invoke({
        "query": query_to_check,
        "dialect": db.dialect,
    })

tools = [
    QuerySQLDatabaseTool,
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
]

#### Routing

### Function to get agent with provided llm model

In [23]:
def format_to_ollama_chat_messages(intermediate_steps):
    agent_scratchpad = []
    
    for step in intermediate_steps:
        agent_scratchpad.append(
            AIMessage(
                content=f"I use this tool \"{step[0].tool}\" " +
                f"with this input\nInput: {step[0].tool_input}"
            )
        )
        if step[0].tool == "QuerySQLDataBaseTool":
            human_message = "This is similar to the database " +\
                "response. Maybe there's something I need there. " +\
                "I can't see it, please help me understand, if " +\
                "there is an answer to my question, please give it " +\
                "to me, use it for this: \"__conversational_response\""
        elif step[0].tool == "InfoSQLDatabaseTool":
            human_message = "It looks like the structure of the database " +\
                "tables(s). Maybe this will help you write the right query? " +\
                "Please make sure you remember the names!"
        elif step[0].tool == "ListSQLDatabaseTool":
            human_message = "It looks like a list of table names. " +\
                "Maybe it will help to get an answer. I think if you don't " +\
                "have enough information, you can use this: \"InfoSQLDatabaseTool\""
        elif step[0].tool == "QuerySQLCheckerTool":
            human_message = "It looks like an SQL query! I think it's the " +\
                "right one. It's worth running it and seeing what it gives, " +\
                "maybe there's an answer to my question. Help me, please!"
        
        agent_scratchpad.append(
            HumanMessage(
                content="I can help you execute the tool. Give me a second... " +
                    "And so, I think I managed to call it, but I can't read " +
                    "what's written there, only a smart AI can understand it." +
                    f"\nI checked, the result of the tool\n{human_message}" +
                    f"\n\nResult of tool: \n{step[1]}"
            )
        )
        
    return agent_scratchpad

def get_agent(llm: OllamaFunctions, functions: List[BaseModel]) -> AgentExecutor:
    """Create and return agent with provided `llm` model"""

    llm_with_tools = llm.bind_tools(tools=functions)
    llm_with_tools.bind(verbose=True)
    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda steps: format_to_ollama_chat_messages(steps['intermediate_steps'])
        )
        | prompt_generator.get_prompt()
        | llm_with_tools
        | OpenAIFunctionsAgentOutputParser()
    )
    
    return agent

In [27]:
agent = get_agent(llama3__with_functions, functions)

def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent.invoke({
            "dialect": "PostgreSQL",
            "input": TestData.QUESTIONS[0],
            "top_k": "3", 
            "intermediate_steps": intermediate_steps
        }, config=RunnableConfig(callbacks=[ConsoleCallbackHandler()]))
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "QuerySQLDataBaseTool": QuerySQLDatabaseTool, 
            "InfoSQLDatabaseTool": InfoSQLDatabaseTool,
            "ListSQLDatabaseTool": ListSQLDatabaseTool, 
            "QuerySQLCheckerTool": QuerySQLCheckerTool,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

-----
## Testing an agent with different models and save the results

In [25]:
print(TestData.ANSWER[0])

Here are the names you asked for: 'Alice', 'Bob', 'Charlie', 'Christofer', 'David'


In [28]:
res = run_agent("")

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "dialect": "PostgreSQL",
  "input": "Print the first 5 names of the passengers, sorted by name",
  "top_k": "3",
  "intermediate_steps": []
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "dialect": "PostgreSQL",
  "input": "Print the first 5 names of the passengers, sorted by name",
  "top_k": "3",
  "intermediate_steps": []
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "dialect": "PostgreSQL",
  "input": "Print the first 5 names of the passengers, sorted by name",
  "top_k": "3",
  "intermediate_steps": []
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "dialect": "PostgreSQL",
  "input": "Print the

In [ ]:
print(res)